In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from openai import OpenAI

client = OpenAI(api_key=api_key)

In [4]:
import json
from tqdm import tqdm

In [ ]:
importance_output_path = "./data/enriched_requirements_importance.json"

In [ ]:
# 프롬프트 생성 함수 (detailed_description 포함)
def generate_prompt(description, detailed_description, module):
    return f"""
당신은 소프트웨어 요구사항을 분석하여 난이도를 판단하는 전문가입니다.

다음은 시스템에 대한 요구사항입니다:

[요구사항 설명]
{description}

[상세 설명]
{detailed_description}

[담당 모듈]
{module}

요구사항을 분석한 뒤, 다음 두 가지 항목을 평가하세요:

**난이도**: 요구사항의 기술적 구현, 연동, 유지보수의 복잡성을 고려하여 '상', '중', '하' 중 하나로 평가하세요.
- 상 (H, High)  : 자체 개발 일정에 차질이 있을 정도의 노력이 필요한 경우
- 중 (M, Medium): 자체 개발 일정 내에 완수 할 수 있는 경우
- 하 (L, Low)   : 자체 개발 일정에 일부분으로 흡수되어도 되는 경우

아래와 같이 **정확히 이 형식**으로만 출력하세요 (불필요한 설명 없이):

난이도: <상|중|하>
"""

In [ ]:
# GPT 응답 처리 함수
def get_importance_and_difficulty(description, detailed_description, module):
    prompt = generate_prompt(description, detailed_description, module)
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "당신은 소프트웨어 분석 전문가입니다."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2
        )
        content = response.choices[0].message.content
        importance = next((line.split(":")[1].strip() for line in content.splitlines() if "중요도" in line), "중")
        difficulty = next((line.split(":")[1].strip() for line in content.splitlines() if "난이도" in line), "중")
        return importance, difficulty
    except Exception as e:
        print("Error:", e)
        return "Error", "Error"


In [ ]:
# JSON 불러오기
with open(importance_output_path, "r", encoding="utf-8") as f:
    requirements = json.load(f)

# 평가 수행
for req in tqdm(requirements):
    desc = req.get("description", "")
    detailed_desc = req.get("detailed_description", "")
    module = req.get("responsible_module", "")
    importance, difficulty = get_importance_and_difficulty(desc, detailed_desc, module)
    req["difficulty"] = difficulty

difficulty_output_path = "./data/enriched_requirements_difficulty.json"

# 결과 저장
with open(difficulty_output_path, "w", encoding="utf-8") as f:
    json.dump(requirements, f, ensure_ascii=False, indent=4)

print("📊 평가 완료: enriched_requirements_difficulty.json 저장됨.")